In [6]:
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

import utils
from utils import SessionStatus, classification_accuracy
from network import NeuralNetwork
import functions as F
import optimizers as Opt

In [7]:
# args parser
#------------------
# config contains all model and session setup options
#P = utils.Parser()
#my_args = '-a softplus'
#config = P.parse_args_from_jupyter_notebook()
#P.print_args()

In [8]:
''' Now you have a CV trainer:
- Pick 4 random seeds all models train on
- Pick your K for K folds
- Pick vars you want to search for
  - Channels : A LOT HERE
    - depth
    - kernel sizes
  - activation?

Default maybe just 1000-2000 iters, SGD, sigmoid, Softmax CE

- then run in double-for loop over:
  - RNG seeds
  - model parameter candidates

Save all results, average out rng-seed dims,
plot, and evaluate what's next
'''

" Now you have a CV trainer:\n- Pick 4 random seeds all models train on\n- Pick your K for K folds\n- Pick vars you want to search for\n  - Channels : A LOT HERE\n    - depth\n    - kernel sizes\n  - activation?\n\nDefault maybe just 1000-2000 iters, SGD, sigmoid, Softmax CE\n\n- then run in double-for loop over:\n  - RNG seeds\n  - model parameter candidates\n\nSave all results, average out rng-seed dims,\nplot, and evaluate what's next\n"

In [9]:
import primesieve
kernels = primesieve.n_primes(40)[3:]
kernels.extend([2**i for i in range(2, 9)])
kernels.extend(np.random.randint(50, 151, 10))
kernels = np.random.permutation(kernels)


[ 31  41  32 151  16  53  78 157  47 149 144 121  11 256  13 163  55 103
 128 124   4  59 131  97   8  61  29 101  83  17 113 167 127 101 130  37
 139   7  64  79 109  71  43  80  67 101  19  23  74  73 107  89 173 137]


In [11]:
kernels

array([ 31,  41,  32, 151,  16,  53,  78, 157,  47, 149, 144, 121,  11,
       256,  13, 163,  55, 103, 128, 124,   4,  59, 131,  97,   8,  61,
        29, 101,  83,  17, 113, 167, 127, 101, 130,  37, 139,   7,  64,
        79, 109,  71,  43,  80,  67, 101,  19,  23,  74,  73, 107,  89,
       173, 137])

In [10]:
#==============================================================================
# Model constants
#==============================================================================
# Model config
#------------------
optimizer  = Opt.SGD
objective  = F.SoftmaxCrossEntropy
#dropout    = config.dropout

# Dataset
#-------------------
X_train = np.load('X_train.ign.npy')

In [ ]:
#==============================================================================
# Model variables
#==============================================================================
#======= activation funcs
activations = [F.Sigmoid, F.SeLU]
#======= RNG seeds
SEEDS = [3310, 99467, 27189, 77771]


k_in  = 4
k_out = 3
#==== simple list of primes, powers of 2, and some rand ints to make up channels
kernels = np.array([ 31,  41,  32, 151,  16,  53,  78, 157,  47, 149, 144, 121,  11,
                     256,  13, 163,  55, 103, 128, 124,   4,  59, 131,  97,   8,  61,
                      29, 101,  83,  17, 113, 167, 127, 101, 130,  37, 139,   7,  64,
                      79, 109,  71,  43,  80,  67, 101,  19,  23,  74,  73, 107,  89,
                      173, 137])

In [ ]:
K = 

In [4]:
trainer = utils.Trainer(channels, optimizer, objective, activation, num_iters, batch_size)

In [5]:
trainer()

#==============================================================================
# Model Summary: 

NeuralNetwork
  Layers: 
     1 : Dense (4, 64)
          : Sigmoid
     2 : Dense (64, 3)

- OPTIMIZER : SGD
- OBJECTIVE : SoftmaxCrossEntropy

# Training results, 2000 iterations
#------------------------------
            Error  |  Accuracy
* Average: 0.62794 | 0.83819
*  Median: 0.62261 | 0.83333
#------------------------------

# Test results, 30 samples
#------------------------------
            Error  |  Accuracy
* Average: 0.44890 | 0.93333
*  Median: 0.49447 | 1.00000
#------------------------------
#==============================================================================
